In [1]:
import ast

In [75]:
data = '''def sum(a,b):
    plus = a+b
    return plus
'''

data

'def sum(a,b):\n    plus = a+b\n    return plus\n'

In [104]:
generated_data = ['def','f','(','target',',',
 'array',')', ':','NEW_LINE','INDENT','count','=','0','NEW_LINE','for','str','in','array',':',
 'NEW_LINE','INDENT','if','target','==','str',':','NEW_LINE','INDENT','count','+=','1',
 'NEW_LINE','DEDENT','DEDENT','return','count','NEW_LINE','DEDENT','NEW_LINE']

generated_data = [ d.replace("NEW_LINE","\n").replace("INDENT","\t") for d in generated_data]
data2 = ' '.join(generated_data)
data2

'def f ( target , array ) : \n \t count = 0 \n for str in array : \n \t if target == str : \n \t count += 1 \n \t \t return count \n \t \n'

In [106]:
def f(target, array):
    count = 0
    for str in array:
            if target == str:
                    count+=1
    return count

In [105]:
tree = ast.parse(data2)

tree

IndentationError: unindent does not match any outer indentation level (<unknown>, line 3)

In [48]:
class C(ast.NodeVisitor):
    def generic_visit(self, node):
        print(type(node).__name__)
        ast.NodeVisitor.generic_visit(self, node)
        
    def visit_Name(self, node):
        print("Name: ", node.id)
        

c = C()
c.visit(tree)

Module
FunctionDef
arguments
arg
arg
Assign
Name:  plus
BinOp
Name:  a
Add
Name:  b
Return
Name:  plus


In [49]:
ast.dump(tree, include_attributes=False)

"Module(body=[FunctionDef(name='sum', args=arguments(posonlyargs=[], args=[arg(arg='a', annotation=None, type_comment=None), arg(arg='b', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='plus', ctx=Store())], value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())), type_comment=None), Return(value=Name(id='plus', ctx=Load()))], decorator_list=[], returns=None, type_comment=None)], type_ignores=[])"

In [51]:
from _ast import AST
import ast
import json

def ast2json(node):
    
    if not isinstance(node, AST):
        raise TypeError( 'expected AST, got %r' % node.__class__.__name__ )
        
    def _format( node ):
        if isinstance(node, AST):
            fields = [('_PyType', _format( node.__class__.__name__ ))]
            fields += [( a, _format(b)) for a, b in iter_fields( node )]
            return '{ %s }' % ', '.join(( '"%s": %s' % field for field in fields))

        if isinstance( node, list ):
            return '[ %s ]' % ', '.join([ _format( x ) for x in node ])

        return json.dumps( node )
    
    return _format( node )

def iter_fields(node):
    for field in node._fields:
        try:
            yield field, getattr( node, field )
        except AttributeError:
            pass

In [52]:
datap = ast.parse(data)
datap

In [57]:
a2j = ast2json(datap)
print(a2j)

{ "_PyType": "Module", "body": [ { "_PyType": "FunctionDef", "name": "sum", "args": { "_PyType": "arguments", "posonlyargs": [  ], "args": [ { "_PyType": "arg", "arg": "a", "annotation": null, "type_comment": null }, { "_PyType": "arg", "arg": "b", "annotation": null, "type_comment": null } ], "vararg": null, "kwonlyargs": [  ], "kw_defaults": [  ], "kwarg": null, "defaults": [  ] }, "body": [ { "_PyType": "Assign", "targets": [ { "_PyType": "Name", "id": "plus", "ctx": { "_PyType": "Store" } } ], "value": { "_PyType": "BinOp", "left": { "_PyType": "Name", "id": "a", "ctx": { "_PyType": "Load" } }, "op": { "_PyType": "Add" }, "right": { "_PyType": "Name", "id": "b", "ctx": { "_PyType": "Load" } } }, "type_comment": null }, { "_PyType": "Return", "value": { "_PyType": "Name", "id": "plus", "ctx": { "_PyType": "Load" } } } ], "decorator_list": [  ], "returns": null, "type_comment": null } ], "type_ignores": [  ] }


In [60]:
a2j = json.loads(a2j)

In [68]:
a2j['body'][0]['body']

[{'_PyType': 'Assign',
  'targets': [{'_PyType': 'Name', 'id': 'plus', 'ctx': {'_PyType': 'Store'}}],
  'value': {'_PyType': 'BinOp',
   'left': {'_PyType': 'Name', 'id': 'a', 'ctx': {'_PyType': 'Load'}},
   'op': {'_PyType': 'Add'},
   'right': {'_PyType': 'Name', 'id': 'b', 'ctx': {'_PyType': 'Load'}}},
  'type_comment': None},
 {'_PyType': 'Return',
  'value': {'_PyType': 'Name', 'id': 'plus', 'ctx': {'_PyType': 'Load'}}}]